# Import Needed Libraries

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

### Read data and store it in dataframe

In [2]:
train_data_dic = '/kaggle/input/brain-tumor-mri-dataset/Training'
imagePaths = []
labels =[]

floders = os.listdir(train_data_dic)

for floder in floders:
    floderPath = os.path.join(train_data_dic, floder)
    imageList = os.listdir(floderPath)
    
    for image in imageList:
        imagePath = os.path.join(floderPath, image)
        
        imagePaths.append(imagePath)
        labels.append(floder)
        
        
# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

train_df = pd.concat([Iseries, Lseries], axis= 1)

In [3]:
train_df

,imagepaths,labels
0,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
...,...,...
5707,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5708,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5709,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5710,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma


In [4]:
test_data_dic = '/kaggle/input/brain-tumor-mri-dataset/Testing'
imagePaths = []
labels =[]

floders = os.listdir(test_data_dic)

for floder in floders:
    floderPath = os.path.join(test_data_dic, floder)
    imageList = os.listdir(floderPath)
    
    for image in imageList:
        imagePath = os.path.join(floderPath, image)
        
        imagePaths.append(imagePath)
        labels.append(floder)
        
        
# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

ts_df = pd.concat([Iseries, Lseries], axis= 1)

In [5]:
ts_df

,imagepaths,labels
0,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
...,...,...
1306,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1307,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1308,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1309,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma


### Split dataframe into train, valid, and test

In [6]:
test_df , valid_df = train_test_split(ts_df, test_size=0.5, shuffle=True, random_state=123)

### Create image data generator

In [7]:
img_size = (224, 224)
batch_size = 16

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

valid_gen = ts_gen.flow_from_dataframe(valid_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

test_gen = ts_gen.flow_from_dataframe(test_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

Found 5712 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.


# Model Structure

In [8]:
img_shape = (224, 224, 3)
class_count = 4

base_model = tf.keras.applications.EfficientNetB4(
            include_top=False,
            weights="imagenet",
            input_shape=img_shape,
            pooling='max'
            )

model = Sequential([
    base_model,
    Dense(256, activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb4 (Functional)     │ ?                      │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,673,823 (67.42 MB)

 Trainable params: 17,548,616 (66.94 MB)

 Non-trainable params: 125,207 (489.09 KB)

In [9]:
epochs = 10 

history = model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

Epoch 1/10


I0000 00:00:1730106529.291359      65 service.cc:145] XLA service 0x7ff2ac0247a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730106529.291426      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1730106643.013891      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


357/357 ━━━━━━━━━━━━━━━━━━━━ 248s 199ms/step - accuracy: 0.7595 - loss: 1.4629 - val_accuracy: 0.8034 - val_loss: 0.6149
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 171ms/step - accuracy: 0.9386 - loss: 0.1661 - val_accuracy: 0.9634 - val_loss: 0.1056
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9723 - loss: 0.0900 - val_accuracy: 0.9848 - val_loss: 0.0445
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9853 - loss: 0.0490 - val_accuracy: 0.9924 - val_loss: 0.0181
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9873 - loss: 0.0331 - val_accuracy: 0.9893 - val_loss: 0.0346
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9835 - loss: 0.0499 - val_accuracy: 0.9924 - val_loss: 0.0145
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9913 - loss: 0.0291 - val_accuracy: 0.9863 - val_loss: 0.0340
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 61s 170ms/step - accuracy: 0.9911 - loss: 0.0315 - va

# Evaluate model

In [10]:
train_score = model.evaluate(train_gen, verbose= 1)
valid_score = model.evaluate(valid_gen, verbose= 1)
test_score = model.evaluate(test_gen, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

357/357 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.9995 - loss: 8.4101e-04
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9984 - loss: 0.0059
41/41 ━━━━━━━━━━━━━━━━━━━━ 10s 240ms/step - accuracy: 0.9914 - loss: 0.0372
Train Loss:  0.0011237376602366567
Train Accuracy:  0.9994747638702393
--------------------
Validation Loss:  0.013826610520482063
Validation Accuracy:  0.9954268336296082
--------------------
Test Loss:  0.05953894555568695
Test Accuracy:  0.9923664331436157
